## XGBoost version

In [ ]:
import xgboost as xgb

In [ ]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
import logging
from sklearn.utils import shuffle

data_path = '../../data'
#train1 = pd.read_csv(os.path.join(data_path, 'invite_info_0926.txt'), header=None, sep='\t')
test1 = pd.read_csv(os.path.join(data_path, 'invite_info_evaluate_1_0926.txt'), header=None, sep='\t')
data = pd.read_hdf('data_dropout.h5',key='data').reset_index()
print(data.info())


'''修改'''

# add: question_basic_extra_feats
# question_info1 = pd.read_hdf('question_info1.hd5', key='data')
# data = pd.merge(data, question_info1, left_on='qid', right_on='问题id', how='left').drop('问题id', axis=1)
# data['diff_qi_days'] = data['day'] - data['q_day']
# data['diff_qi_hours'] = data['hour'] - data['q_hour']
# data.loc[:,'diff_qi_hours'] += 23
# drop_feat = ['q_day','q_hour']
# data  = data.drop(drop_feat, axis=1)
# print(data.info())

# add: user_question_topic_embedding_similarity_feats
import pickle
with open('test_uid_qid_cosine_sim.pkl', 'rb') as file:
    test_cosine_sim = pickle.load(file).drop_duplicates(subset=['uid','qid'], keep='first', inplace=False)
with open('train_label_uid_qid_cosine_sim.pkl', 'rb') as file:
    train_cosine_sim = pickle.load(file).drop_duplicates(subset=['uid','qid'], keep='first', inplace=False)
train_sim = pd.merge(data[:1506141], train_cosine_sim, on=['uid','qid'], how='left')
test_sim = pd.merge(data[1506141:], test_cosine_sim, on=['uid','qid'], how='left')
data = pd.concat([train_sim, test_sim], axis=0, sort=True)  
del train_sim, test_sim

#add: dnn_score_output
# dnn_results = pd.read_csv('submit_auto_drop.txt', sep='\t')
# dnn_results.columns = ['qid','uid','i_time','dnn_score']
# dnn_results['day'] = dnn_results['i_time'].apply(lambda x:x.split('-')[0].split('D')[1]).astype(int)
# dnn_results['hour'] = dnn_results['i_time'].apply(lambda x:x.split('-')[1].split('H')[1]).astype(int)
# del dnn_results['i_time']
# dnn_results = dnn_results.drop_duplicates(subset=['uid','qid','day','hour'], keep='first', inplace=False)
# data = pd.merge(data, dnn_results, on=['uid','qid','day','hour'], how='left')

# add: user_answer_side_feats
user_answer_feats_val = pd.read_hdf('../user_answer_val.h5',key='data')
user_answer_feats_train = pd.read_hdf('../user_answer_train.h5',key='data')
user_answer_feats_train = user_answer_feats_train[['用户id','u_a_i_diffhour_mean','u_a_i_diffhour_sum',
                                                   'u_a_i_diffhour_max','u_a_i_diffhour_min','u_a_i_diffday_mean',
                                                   'u_a_i_diffday_sum','u_a_i_diffday_max','u_a_i_diffday_min',
                                                   '用户习惯回答时间-hour','用户最近回答数-3天','用户最近回答数-7天','用户最近回答数-14天',
                                                   '用户习惯回答时间-wkday']]#,'用户上次回答时间-day']]#,
user_answer_feats_val = user_answer_feats_val[['用户id','u_a_i_diffhour_mean','u_a_i_diffhour_sum',
                                                   'u_a_i_diffhour_max','u_a_i_diffhour_min','u_a_i_diffday_mean',
                                                   'u_a_i_diffday_sum','u_a_i_diffday_max','u_a_i_diffday_min',
                                                   '用户习惯回答时间-hour','用户最近回答数-3天','用户最近回答数-7天','用户最近回答数-14天',
                                              '用户习惯回答时间-wkday']]#,'用户上次回答时间-day']]#' ,

user_answer_feats_train = user_answer_feats_train.rename(columns={'用户习惯回答时间-hour':'u_most_a_hour','用户最近回答数-3天':'u_a_num_latest3',
                                        '用户最近回答数-7天':'u_a_num_latest7','用户最近回答数-14天':'u_a_num_latest14',
                                        '用户习惯回答时间-wkday':'u_most_a_wkday','用户上次回答时间-day':'u_last_a_time'})#,
user_answer_feats_val = user_answer_feats_val.rename(columns={'用户习惯回答时间-hour':'u_most_a_hour','用户最近回答数-3天':'u_a_num_latest3',
                                        '用户最近回答数-7天':'u_a_num_latest7','用户最近回答数-14天':'u_a_num_latest14',
                                        '用户习惯回答时间-wkday':'u_most_a_wkday','用户上次回答时间-day':'u_last_a_time'})#

#print(user_answer_feats_train[['用户id','u_last_a_time']])

train2 = pd.merge(data[:1506141], user_answer_feats_train, left_on='uid', right_on='用户id', how='left' ).drop('用户id',axis=1)
test2 = pd.merge(data[1506141:],user_answer_feats_val, left_on='uid', right_on='用户id', how='left' ).drop('用户id',axis=1)
#train2['u_last_a_time'] = train2['day'] - train2['u_last_a_time']
#test2['u_last_a_time'] = test2['day'] - test2['u_last_a_time']

data = pd.concat([train2, test2], axis=0, sort=True)
del train2, test2

# add: user_topic_side_feats
user_topic_feats = pd.read_hdf('../user_topic_feat.h5',key='data')
data = pd.merge(data,user_topic_feats, left_on='uid', right_on='author_id',how='left').drop('author_id',axis=1)
print("loaded topic feats")

user_question_topic = pd.read_hdf('../member_question_feat.h5', key='data')
user_question_topic = user_question_topic.rename(columns={'用户问题的话题关注重叠数':'u_q_topic_att_num','用户问题的话题感兴趣重叠数':'u_q_topic_ints_num'})

data = pd.merge(data, user_question_topic, left_on=['uid','qid'], right_on=['author_id','question_id'], how='left').drop(['author_id','question_id'],axis=1)
print("loaded user_question topics")
#user_qa_days_feats_train = pd.read_hdf('../u_qa_day_feats_train.h5',key='data')
#user_qa_days_feats_val = pd.read_hdf('../u_qa_day_feats_val.h5',key='data')
#print("loaded u_qa_days")

'''修改end'''

print(data.head(3))

'''mmm'''
feature_cols = [x for x in data.columns if x not in ('index','uid', 'qid','dropoutrate', 'label', 'dt', 'day', 'q_inv_mean', 'q_inv_sum', 'q_inv_std', 'q_inv_count')]
'''end'''

print(feature_cols)


# #对负样本采样
# train_all_pos = train_all[train_all['label']==1]
# train_all_neg = train_all[train_all['label']==0].sample(frac=0.5)
# train_all = shuffle(pd.concat([train_all_pos, train_all_neg], axis=0, sort=True))
# print("shuffled")
# del train_all_pos, train_all_neg

train_all = data.iloc[:1506141]
train_all = shuffle(train_all)
X_train_all = train_all[feature_cols]
y_train_all = train_all['label']
test = data.iloc[1506141:]


del data
print(len(test))
assert len(test) == test1.shape[0]



In [ ]:
logging.info("train shape %d, test shape %s", 1506141, test.shape)

fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores=[]

for index, (train_idx, val_idx) in enumerate(fold.split(X=X_train_all, y=y_train_all)):
    break
X_train, X_val, y_train, y_val = X_train_all.iloc[train_idx][feature_cols], X_train_all.iloc[val_idx][feature_cols], \
                                 y_train_all.iloc[train_idx], \
                                 y_train_all.iloc[val_idx]


model_xgb = xgb.XGBClassifier(max_depth=6, n_estimators=2000, seed=1000, silent=True)

    
model_xgb.fit(X_train, y_train,
              eval_metric='auc',
              eval_set=[(X_val, y_val)],
              verbose=True,
              early_stopping_rounds=1000)
    
    
#     score = model_xgb.best_score_['eval']['auc']
#     scores.append(score)
#     print('fold %d round %d : score: %.6f | mean score %.6f' % (index+1, model_xgb.best_iteration_, score, np.mean(scores))) 
   
#     y_pred = model_xgb.predict_proba(test[feature_cols])[:,1]
#     np.savetxt('y_pred%d.txt'%(index+1),y_pred)
#     print('saved')

del X_train_all

sub = test1.copy()
sub_size = len(sub)
sub['label'] = model_xgb.predict_proba(test[feature_cols])[:, 1]


sub.to_csv('result_xgb.txt', index=None, header=None, sep='\t')

pd.set_option('display.max_rows', None)
print(pd.DataFrame({
    'column': feature_cols,
    'importance': model_xgb.feature_importances_
}).sort_values(by='importance', ascending=False))

model_xgb.save_model('xgb_model.txt')

In [ ]:
test1 = pd.read_csv(os.path.join(data_path, 'invite_info_evaluate_1_0926.txt'), header=None, sep='\t')
data = pd.read_hdf('data_dropout.h5',key='data').reset_index()
sub = pd.concat(data, test1, axis=0)
sub_size = len(sub)
sub['label'] = model_xgb.predict_proba(test[feature_cols])[:, 1]